In [1]:
%%writefile plotkernel.py

from ipykernel.kernelbase import Kernel
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
import urllib, base64

Writing plotkernel.py


In [2]:
%%writefile plotkernel.py -a

def _to_png(fig):
    """Return a base64-encoded PNG from a
    matplotlib figure."""
    imgdata = BytesIO()
    fig.savefig(imgdata, format='png')
    imgdata.seek(0)
    return urllib.parse.quote(
        base64.b64encode(imgdata.getvalue()))

Appending to plotkernel.py


In [3]:
%%writefile plotkernel.py -a

space_for_numpy = {n: getattr(np, n)
                    for n in dir(np)}
def _parse_function(code):
    """Return a NumPy function from a
    string 'y=f(x)'."""
    return lambda x: eval(code.split('=')[1].strip(),
                          _numpy_namespace, {'x': x})

Appending to plotkernel.py


change:
_numpy_namespace -> space_for_numpy

In [4]:
%%writefile plotkernel.py -a

class PlotKernel(Kernel):
    implementation = 'Plot'
    implementation_version = '1.0'
    language = 'python'  # will be used for
                         # syntax highlighting
    language_version = '3.6'
    language_info = {'name': 'plotter',
                     'mimetype': 'text/plain',
                     'extension': '.py'}
    banner = "Simple plotting"

Appending to plotkernel.py


In [17]:
%%writefile plotkernel.py -a

    def do_execute(self, code, silent,
                   store_history=True,
                   user_expressions=None,
                   allow_stdin=False):
 fig, ax = plt.subplots(1, 1, figsize=(6,4),
                               dpi=100)
        x = np.linspace(-5., 5., 200)
        functions = code.split('\n')
        for fun in functions:
            f = _parse_function(fun)
            y = f(x)
            ax.plot(x, y)
        ax.set_xlim(-5, 5)
 png = _to_png(fig)

        if not silent:
             self.send_response(
                self.iopub_socket,
                'stream', {
                    'name': 'stdout',
                    'data': ('Plotting {n} '
                             'function(s)'). \
                            format(n=len(functions))})
             content = {
                'source': 'kernel',
            'data': {
                    'image/png': png
                }, 'metadata' : {
                      'image/png' : {
                        'width': 600,
                        'height': 400
                      }
                    }
            }
             self.send_response(self.iopub_socket,
                'display_data', content)
    return {'status': 'ok',
                'execution_count':
                    self.execution_count,
                'payload': [],
                'user_expressions': {},
               }

Appending to plotkernel.py


In [18]:
%%writefile plotkernel.py -a

if __name__ == '__main__':
    from ipykernel.kernelapp import IPKernelApp
    IPKernelApp.launch_instance(
        kernel_class=PlotKernel)

Appending to plotkernel.py


In [19]:
%mkdir -p plotter/
%%writefile plotter/kernel.json
{
 "argv": ["python", "-m",
          "plotkernel", "-f",
          "{connection_file}"],
 "display_name": "Plotter",
 "name": "Plotter",
 "language": "python"
}

Syntaxfehler.
UsageError: Line magic function `%%writefile` not found.


In [15]:
!jupyter kernelspec install --user plotter

Traceback (most recent call last):
  File "C:\Users\minhd\anaconda3\Scripts\jupyter-kernelspec-script.py", line 10, in <module>
    sys.exit(KernelSpecApp.launch_instance())
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\minhd\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\minhd\anaconda3\Lib\site-packages\jupyter_client\kernelspecapp.py", line 337, in start
    return self.subapp.start()
           ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\minhd\anaconda3\Lib\site-packages\jupyter_client\kernelspecapp.py", line 152, in start
    self.kernel_spec_manager.install_kernel_spec(
  File "C:\Users\minhd\anaconda3\Lib\site-packages\jupyter_client\kernelspec.py", line 406, in install_kernel_spec
    shutil.copytree(source_dir, destination)
  File "C:\Users\minhd\anaconda3\Lib\shutil.py", line 559, in copytree
    with os.scandir(src) as itr:
         ^^^^^^^^^^^^^^^
FileNotFoundError: [WinError 3] Das System

In [20]:
!jupyter kernelspec list

Available kernels:
  python3    C:\Users\minhd\anaconda3\share\jupyter\kernels\python3


0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
